# Reference

* http://neuralnetworksanddeeplearning.com/

In [2]:
import numpy as np

In [3]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [4]:
def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

In [5]:
a = np.random.random((3, 4))

In [6]:
np.random.randn(3, 4)

array([[ 1.82417516, -0.2403351 , -0.64042118, -1.6091103 ],
       [-0.37115847, -1.58796884, -1.74194645,  0.75002029],
       [-1.25362636, -0.09506361,  2.42418629,  1.23684067]])

In [7]:
import numpy as np
import random

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))


class Network():
    def __init__(self, sizes: list[int]):
        '''
        sizes: specifies the number of neurons each layer
        '''
        # [0, 1] uniformly random initialization is not a good idea, because you weights are all positive numbers 
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.num_layers = len(sizes)
        self.sizes = sizes
    
    def forward(self, x):
        for w, b in zip(self.weights, self.biases):
            x = sigmoid(np.dot(w, x) + b)
        return x


    def SGD(self, training_data, epochs, batch_size, lr, test_data=None):
        '''
        train the neutral network with SGD
        training_data: a list of tuple of (x, y) where x is of shape (size[0], 1) and y is of shape(size[-1], 1)
        epochs: the number of epochs of training
        batch_size: the mini-batch size
        lr: learning rate
        test_data: if provided, we will also evaluate on the test_data after each epoch
        '''
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+batch_size] for k in range(0, n, batch_size)]
            for mini_batch in mini_batches:
                self.update_minibatch(mini_batch, lr)
            if test_data:
                correct_cnt = self.evaluate(test_data)
                print(f"Epoch {epoch}, classification accuracy is {correct_cnt} / {n_test}")
            
        
    
    def update_minibatch(self, minibatch, lr):
        '''
        Update the weights and bias after backprop for each minibatch
        '''
        weight_updates = [np.zeros(weight.shape) for weight in self.weights]
        bias_updates = [np.zeros(bias.shape) for bias in self.biases]
        for x, y in minibatch:
            weight_deltas, bias_deltas = self.backprop(x, y)
            weight_updates = [w_update + w_delta for w_update, w_delta in zip(weight_updates, weight_deltas)]
            bias_updates = [b_update + b_delta for b_update, b_delta in zip(bias_updates, bias_deltas)]
        
        self.weights = [w - lr * weight_update / len(minibatch) for w, weight_update in zip(self.weights, weight_updates)]
        self.biases = [b - lr * bias_update / len(minibatch) for b, bias_update in zip(self.biases, bias_updates)]
        
    
    def backprop(self, x, y):
        '''
        x: the input of the neutral network of shape [size[0], 1]
        y: the label of size [size[-1], 1]
        It only deals with one training example. The aggregation will be handled in update_mini_batch()
        First do a feed forward pass to get the prediction and then do the backpropagation layer by layer
        return the weights and bias updates layer by layer
        '''
        # we need to store the activations to get the Jacobian of neutrons of next layer wrt to weights
        # used to get derivative wrt weights
        activations = [x]
        # we also need to store the values before activation. This is used to get the derivative wrt to activation functions
        # like sigmoid's derivative is sigmoid(x) * (1-sigmoid(x)) where x is the value before applying sigmoid activation
        zs = []
        activation = x
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # the partial derivative wrt the activations of shape [size[-1], 1], it will be passed down
        partial_deriv_activation = self.cost_deriv(activation, y) * sigmoid_prime(zs[-1])
        weights_deltas = [np.zeros(w.shape) for w in self.weights]
        bias_deltas = [np.zeros(b.shape) for b in self.biases]
        # there are self.num_layers - 1 weights and bias
        for i in range(1, self.num_layers):
            # first matrix multiplication in backprop at each step, partial_deriv_activation * Jacobaian wrt weights
            weights_deltas[-i] = np.dot(partial_deriv_activation, activations[-(i+1)].transpose())
            bias_deltas[-i] = partial_deriv_activation
            # second matrix multiplication in backprop at each step, partial_deriv_activation * Jacobian wrt node values in prev layer
            # we can see that this derivative is passed backward and multiplied together
            # this is where vanishing or exploding gradients come from
            if i < self.num_layers-1:
                partial_deriv_activation = np.dot(self.weights[-i].transpose(), partial_deriv_activation) * sigmoid_prime(zs[-(i+1)])
        return (weights_deltas, bias_deltas)
            
    def evaluate(self, test_data) -> int:
        '''
        test_data: a list of tuple (x, y) where x is of shape [size[0], 1] and y is an integer ranging from 0 to 9
        '''
        res = [(np.argmax(self.forward(x)), y) for x, y in test_data]
        return sum([int(x == y) for x, y in res])
        
    # this is the detrivative of mean squared error
    def cost_deriv(self, pred, y):
        return (pred - y)

# MNIST Data Load

In [8]:
import gzip
import pickle


def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    file_path = './data/mnist.pkl.gz'
    with gzip.open(file_path, 'rb') as f:
        # the default encoding is ascii, we need to use latin1 to avoid error when decoding
        training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    train_data, val_data, test_data = load_data()
    train_data_input = [x.reshape(784, 1) for x in train_data[0]]
    train_data_labels = [one_hot_encoding(y) for y in train_data[1]]
    train_data = list(zip(train_data_input, train_data_labels))
    val_data_input = [x.reshape(784, 1) for x in val_data[0]]
    val_data = list(zip(val_data_input, val_data[1]))
    test_data_input = [x.reshape(784, 1) for x in test_data[0]]
    test_data = list(zip(test_data_input, test_data[1]))
    return train_data, val_data, test_data

def one_hot_encoding(x: int):
    res = np.zeros((10, 1), dtype='int')
    res[x] = 1
    return res

In [9]:
train_data, val_data, test_data = load_data_wrapper()
# train_data[0][0].shape, train_data[1].shape, val_data[0].shape, val_data[1].shape, test_data[0].shape, test_data[1].shape

In [10]:
train_data[0][0].shape, train_data[0][1].shape, val_data[0][0].shape, val_data[0][1].shape, test_data[0][0].shape, test_data[0][1].shape

((784, 1), (10, 1), (784, 1), (), (784, 1), ())

# Train the Network

With only one hidden layer, the best classification accuracy on the test dataset could be above 90%



In [11]:
network = Network([784, 30, 10])
# hyperparameters
epochs = 10
batch_size = 100
lr = 3

In [12]:
network.SGD(train_data, epochs, batch_size, lr, test_data)

Epoch 0, classification accuracy is 7266 / 10000
Epoch 1, classification accuracy is 7784 / 10000
Epoch 2, classification accuracy is 7951 / 10000
Epoch 3, classification accuracy is 8060 / 10000
Epoch 4, classification accuracy is 8145 / 10000
Epoch 5, classification accuracy is 8899 / 10000
Epoch 6, classification accuracy is 8969 / 10000
Epoch 7, classification accuracy is 9039 / 10000
Epoch 8, classification accuracy is 9086 / 10000
Epoch 9, classification accuracy is 9129 / 10000


# Improved Network
1. Weight initialization with scaled factor of $1/\sqrt{n_{in}}$
2. Weight decay regularization

In [ ]:
import numpy as np
import random

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

def softmax(z):
    return np.exp(z) / np.sum(np.exp(z))


class Network2():
    def __init__(self, sizes: list[int]):
        '''
        sizes: specifies the number of neurons each layer
        '''
        # random normal distribution with standard deviation adjustment
        self.weights = [np.random.randn(y, x) / np.sqrt(x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.num_layers = len(sizes)
        self.sizes = sizes
    
    def forward(self, x):
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            x = sigmoid(np.dot(w, x) + b)
        # use softmax in the final layer activation
        return softmax(np.dot(self.weights[-1], x) + self.biases[-1])


    def SGD(self, training_data, epochs, batch_size, lr, weight_decay= 0.0, test_data=None):
        '''
        train the neutral network with SGD
        training_data: a list of tuple of (x, y) where x is of shape (size[0], 1) and y is of shape(size[-1], 1)
        epochs: the number of epochs of training
        batch_size: the mini-batch size
        lr: learning rate
        test_data: if provided, we will also evaluate on the test_data after each epoch
        '''
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+batch_size] for k in range(0, n, batch_size)]
            for mini_batch in mini_batches:
                self.update_minibatch(mini_batch, lr, n, weight_decay)
            if test_data:
                correct_cnt = self.evaluate(test_data)
                print(f"Epoch {epoch}, classification accuracy is {correct_cnt} / {n_test}")
            
        
    
    def update_minibatch(self, minibatch, lr, n, weight_decay):
        '''
        Update the weights and bias after backprop for each minibatch
        '''
        weight_updates = [np.zeros(weight.shape) for weight in self.weights]
        bias_updates = [np.zeros(bias.shape) for bias in self.biases]
        for x, y in minibatch:
            weight_deltas, bias_deltas = self.backprop(x, y)
            weight_updates = [w_update + w_delta for w_update, w_delta in zip(weight_updates, weight_deltas)]
            bias_updates = [b_update + b_delta for b_update, b_delta in zip(bias_updates, bias_deltas)]
        
        self.weights = [(1-lr * weight_decay / n) * w - lr * weight_update / len(minibatch) for w, weight_update in zip(self.weights, weight_updates)]
        self.biases = [b - lr * bias_update / len(minibatch) for b, bias_update in zip(self.biases, bias_updates)]
        
    
    def backprop(self, x, y):
        '''
        x: the input of the neutral network of shape [size[0], 1]
        y: the label of size [size[-1], 1]
        It only deals with one training example. The aggregation will be handled in update_mini_batch()
        First do a feed forward pass to get the prediction and then do the backpropagation layer by layer
        return the weights and bias updates layer by layer
        '''
        # we need to store the activations to get the Jacobian of neutrons of next layer wrt to weights
        # used to get derivative wrt weights
        activations = [x]
        # we also need to store the values before activation. This is used to get the derivative wrt to activation functions
        # like sigmoid's derivative is sigmoid(x) * (1-sigmoid(x)) where x is the value before applying sigmoid activation
        zs = []
        activation = x
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # the partial derivative wrt the activations of shape [size[-1], 1], it will be passed down
        partial_deriv_activation = (activation - y)
        weights_deltas = [np.zeros(w.shape) for w in self.weights]
        bias_deltas = [np.zeros(b.shape) for b in self.biases]
        # there are self.num_layers - 1 weights and bias
        for i in range(1, self.num_layers):
            # first matrix multiplication in backprop at each step, partial_deriv_activation * Jacobaian wrt weights
            weights_deltas[-i] = np.dot(partial_deriv_activation, activations[-(i+1)].transpose())
            bias_deltas[-i] = partial_deriv_activation
            # second matrix multiplication in backprop at each step, partial_deriv_activation * Jacobian wrt node values in prev layer
            # we can see that this derivative is passed backward and multiplied together
            # this is where vanishing or exploding gradients come from
            if i < self.num_layers-1:
                partial_deriv_activation = np.dot(self.weights[-i].transpose(), partial_deriv_activation) * sigmoid_prime(zs[-(i+1)])
        return (weights_deltas, bias_deltas)
            
    def evaluate(self, test_data) -> int:
        '''
        test_data: a list of tuple (x, y) where x is of shape [size[0], 1] and y is an integer ranging from 0 to 9
        '''
        res = [(np.argmax(self.forward(x)), y) for x, y in test_data]
        return sum([int(x == y) for x, y in res])

In [20]:
network2 = Network2([784, 30, 10])
# hyperparameters
epochs = 30
batch_size = 100
lr = 3
weight_decay = 0.1

In [21]:
network2.SGD(train_data, epochs, batch_size, lr, weight_decay, test_data)

Epoch 0, classification accuracy is 9119 / 10000
Epoch 1, classification accuracy is 9274 / 10000
Epoch 2, classification accuracy is 9301 / 10000
Epoch 3, classification accuracy is 9398 / 10000
Epoch 4, classification accuracy is 9446 / 10000
Epoch 5, classification accuracy is 9406 / 10000
Epoch 6, classification accuracy is 9447 / 10000
Epoch 7, classification accuracy is 9475 / 10000
Epoch 8, classification accuracy is 9447 / 10000
Epoch 9, classification accuracy is 9503 / 10000
Epoch 10, classification accuracy is 9453 / 10000
Epoch 11, classification accuracy is 9509 / 10000
Epoch 12, classification accuracy is 9462 / 10000
Epoch 13, classification accuracy is 9528 / 10000
Epoch 14, classification accuracy is 9469 / 10000
Epoch 15, classification accuracy is 9512 / 10000
Epoch 16, classification accuracy is 9520 / 10000
Epoch 17, classification accuracy is 9514 / 10000
Epoch 18, classification accuracy is 9514 / 10000
Epoch 19, classification accuracy is 9513 / 10000
Epoch 20, 

In [22]:
network2 = Network2([784, 30, 10])
# hyperparameters
epochs = 30
batch_size = 100
lr = 3
weight_decay = 1
network2.SGD(train_data, epochs, batch_size, lr, weight_decay, test_data)

Epoch 0, classification accuracy is 9285 / 10000
Epoch 1, classification accuracy is 9361 / 10000
Epoch 2, classification accuracy is 9299 / 10000
Epoch 3, classification accuracy is 9511 / 10000
Epoch 4, classification accuracy is 9479 / 10000
Epoch 5, classification accuracy is 9536 / 10000
Epoch 6, classification accuracy is 9540 / 10000
Epoch 7, classification accuracy is 9532 / 10000
Epoch 8, classification accuracy is 9525 / 10000
Epoch 9, classification accuracy is 9550 / 10000
Epoch 10, classification accuracy is 9543 / 10000
Epoch 11, classification accuracy is 9560 / 10000
Epoch 12, classification accuracy is 9587 / 10000
Epoch 13, classification accuracy is 9594 / 10000
Epoch 14, classification accuracy is 9574 / 10000
Epoch 15, classification accuracy is 9594 / 10000
Epoch 16, classification accuracy is 9597 / 10000
Epoch 17, classification accuracy is 9565 / 10000
Epoch 18, classification accuracy is 9591 / 10000
Epoch 19, classification accuracy is 9604 / 10000
Epoch 20, 

In [23]:
network2 = Network2([784, 30, 10])
# hyperparameters
epochs = 30
batch_size = 100
lr = 3
weight_decay = 5
network2.SGD(train_data, epochs, batch_size, lr, weight_decay, test_data)

Epoch 0, classification accuracy is 9132 / 10000
Epoch 1, classification accuracy is 9192 / 10000
Epoch 2, classification accuracy is 9278 / 10000
Epoch 3, classification accuracy is 9307 / 10000
Epoch 4, classification accuracy is 9253 / 10000
Epoch 5, classification accuracy is 9382 / 10000
Epoch 6, classification accuracy is 9381 / 10000
Epoch 7, classification accuracy is 9385 / 10000
Epoch 8, classification accuracy is 9435 / 10000
Epoch 9, classification accuracy is 9444 / 10000
Epoch 10, classification accuracy is 9406 / 10000
Epoch 11, classification accuracy is 9513 / 10000
Epoch 12, classification accuracy is 9520 / 10000
Epoch 13, classification accuracy is 9517 / 10000
Epoch 14, classification accuracy is 9535 / 10000
Epoch 15, classification accuracy is 9523 / 10000
Epoch 16, classification accuracy is 9573 / 10000
Epoch 17, classification accuracy is 9532 / 10000
Epoch 18, classification accuracy is 9548 / 10000
Epoch 19, classification accuracy is 9597 / 10000
Epoch 20, 